In [56]:
import pandas as pd
from nba_api.stats.endpoints import boxscorescoringv3, leaguegamefinder,  playoffpicture, commonplayoffseries
import time




In [49]:
teams_list = pd.read_csv("https://gist.githubusercontent.com/Tgemayel/e6e282b9aa538bb8b8b7/raw/f498ceeff06c995afc62ed8c89d634026f954d20/gistfile1.txt", delimiter = "\t")
team_list = teams_list["ATL"].tolist()

In [50]:
recent_games = leaguegamefinder.LeagueGameFinder().get_data_frames()[0]
nba_games = recent_games[recent_games["TEAM_ABBREVIATION"].isin(team_list)]
nba_games

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
4,22024,1610612748,MIA,Miami Heat,1522400076,2024-07-22,MIA vs. MEM,W,213,120,...,0.520,10,24,34,27,7,7,16,33,2.0
5,22024,1610612763,MEM,Memphis Grizzlies,1522400076,2024-07-22,MEM @ MIA,L,215,118,...,0.676,15,23,38,23,9,4,11,24,-2.0
6,22024,1610612753,ORL,Orlando Magic,1522400069,2024-07-21,ORL @ MIN,L,200,100,...,0.960,8,16,24,23,10,6,19,18,-7.0
8,22024,1610612763,MEM,Memphis Grizzlies,1522400068,2024-07-21,MEM vs. LAC,W,200,99,...,0.826,10,20,30,24,10,5,14,19,6.2
10,22024,1610612755,PHI,Philadelphia 76ers,1522400071,2024-07-21,PHI @ BOS,W,200,103,...,0.720,7,19,26,17,8,5,11,21,10.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29991,22019,1610612747,LAL,Los Angeles Lakers,0021900151,2019-11-12,LAL @ PHX,W,240,123,...,0.524,8,36,44,39,4,4,9,21,8.0
29993,22019,1610612754,IND,Indiana Pacers,0021900146,2019-11-12,IND vs. OKC,W,239,111,...,0.923,11,39,50,21,6,4,14,20,26.0
29994,22019,1610612758,SAC,Sacramento Kings,0021900153,2019-11-12,SAC vs. POR,W,239,107,...,0.769,11,35,46,24,9,5,14,25,8.0
29995,22019,1610612755,PHI,Philadelphia 76ers,0021900147,2019-11-12,PHI vs. CLE,W,240,98,...,0.769,11,35,46,24,8,8,14,20,1.0


In [51]:
amount_of_years = 5
years_offset = 19
playoff_games_retainer = dict()

for year in range(amount_of_years):
    try :
    iteration_year = year + years_offset
    playoff_games_retainer["p" + str(iteration_year)] = commonplayoffseries.CommonPlayoffSeries(season = "20" + str(iteration_year) + "-" + str(iteration_year + 1)).get_data_frames()[0]
playoff_games_retainer



{'p19':        GAME_ID  HOME_TEAM_ID  VISITOR_TEAM_ID  SERIES_ID  GAME_NUM
 0   0041900101    1610612749       1610612753  004190010         1
 1   0041900102    1610612749       1610612753  004190010         2
 2   0041900103    1610612753       1610612749  004190010         3
 3   0041900104    1610612753       1610612749  004190010         4
 4   0041900105    1610612749       1610612753  004190010         5
 ..         ...           ...              ...        ...       ...
 78  0041900402    1610612747       1610612748  004190040         2
 79  0041900403    1610612748       1610612747  004190040         3
 80  0041900404    1610612748       1610612747  004190040         4
 81  0041900405    1610612747       1610612748  004190040         5
 82  0041900406    1610612748       1610612747  004190040         6
 
 [83 rows x 5 columns],
 'p20':        GAME_ID  HOME_TEAM_ID  VISITOR_TEAM_ID  SERIES_ID  GAME_NUM
 0   0042000101    1610612755       1610612764  004200010         1
 1   004

In [52]:
seasons_19to24_playoffs = list()
for playoff_series_xx in playoff_games_retainer.values():
    seasons_19to24_playoffs.append(pd.merge(playoff_series_xx, recent_games, on="GAME_ID", how="inner" ))
all_playoffs_games = pd.concat(seasons_19to24_playoffs)

all_playoffs_games.shape

(842, 32)

3
3


In [59]:
team_performance_list = list()

for game_id in all_playoffs_games["GAME_ID"]:
    try:
        team_box_score = boxscorescoringv3.BoxScoreScoringV3(game_id = game_id).get_data_frames()[0]
        team_performance_list.append(team_box_score)
        time.sleep(2)
    except:
        print("you tried but oh well, row lost i guess")
team_performance_df = pd.concat(team_performance_list)

team_performance_df

you tried but oh well, row lost i guess
you tried but oh well, row lost i guess
you tried but oh well, row lost i guess


,gameId,teamId,teamCity,teamName,teamTricode,teamSlug,personId,firstName,familyName,nameI,...,percentagePointsFastBreak,percentagePointsFreeThrow,percentagePointsOffTurnovers,percentagePointsPaint,percentageAssisted2pt,percentageUnassisted2pt,percentageAssisted3pt,percentageUnassisted3pt,percentageAssistedFGM,percentageUnassistedFGM
0,0041900101,1610612753,Orlando,Magic,ORL,magic,203516,James,Ennis III,J. Ennis III,...,0.000,0.273,0.273,0.182,1.0,0.0,1.0,0.0,1.000,0.000
1,0041900101,1610612753,Orlando,Magic,ORL,magic,1629109,Gary,Clark,G. Clark,...,0.200,0.200,0.200,0.000,0.0,0.0,1.0,0.0,1.000,0.000
2,0041900101,1610612753,Orlando,Magic,ORL,magic,202696,Nikola,Vucevic,N. Vucevic,...,0.086,0.000,0.200,0.400,0.7,0.3,1.0,0.0,0.800,0.200
3,0041900101,1610612753,Orlando,Magic,ORL,magic,203095,Evan,Fournier,E. Fournier,...,0.000,0.000,0.667,0.000,0.0,0.0,1.0,0.0,1.000,0.000
4,0041900101,1610612753,Orlando,Magic,ORL,magic,1628365,Markelle,Fultz,M. Fultz,...,0.133,0.133,0.000,0.267,0.2,0.8,0.0,1.0,0.167,0.833
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25,0042300405,1610612738,Boston,Celtics,BOS,celtics,1629004,Svi,Mykhailiuk,S. Mykhailiuk,...,0.000,0.000,0.000,0.000,0.0,0.0,0.0,0.0,0.000,0.000
26,0042300405,1610612738,Boston,Celtics,BOS,celtics,1629674,Neemias,Queta,N. Queta,...,0.000,0.000,0.000,0.000,0.0,0.0,0.0,0.0,0.000,0.000
27,0042300405,1610612738,Boston,Celtics,BOS,celtics,1630531,Jaden,Springer,J. Springer,...,0.000,0.000,0.000,0.000,0.0,0.0,0.0,0.0,0.000,0.000
28,0042300405,1610612738,Boston,Celtics,BOS,celtics,1630214,Xavier,Tillman,X. Tillman,...,0.000,0.000,0.000,0.000,0.0,0.0,0.0,0.0,0.000,0.000


In [62]:
# will switch to player performance cause accidentilly generated wrong dataset
#Note : this is commented for now 
#player_performace_df = team_performance_df
player_performace_df

,gameId,teamId,teamCity,teamName,teamTricode,teamSlug,personId,firstName,familyName,nameI,...,percentagePointsFastBreak,percentagePointsFreeThrow,percentagePointsOffTurnovers,percentagePointsPaint,percentageAssisted2pt,percentageUnassisted2pt,percentageAssisted3pt,percentageUnassisted3pt,percentageAssistedFGM,percentageUnassistedFGM
0,0041900101,1610612753,Orlando,Magic,ORL,magic,203516,James,Ennis III,J. Ennis III,...,0.000,0.273,0.273,0.182,1.0,0.0,1.0,0.0,1.000,0.000
1,0041900101,1610612753,Orlando,Magic,ORL,magic,1629109,Gary,Clark,G. Clark,...,0.200,0.200,0.200,0.000,0.0,0.0,1.0,0.0,1.000,0.000
2,0041900101,1610612753,Orlando,Magic,ORL,magic,202696,Nikola,Vucevic,N. Vucevic,...,0.086,0.000,0.200,0.400,0.7,0.3,1.0,0.0,0.800,0.200
3,0041900101,1610612753,Orlando,Magic,ORL,magic,203095,Evan,Fournier,E. Fournier,...,0.000,0.000,0.667,0.000,0.0,0.0,1.0,0.0,1.000,0.000
4,0041900101,1610612753,Orlando,Magic,ORL,magic,1628365,Markelle,Fultz,M. Fultz,...,0.133,0.133,0.000,0.267,0.2,0.8,0.0,1.0,0.167,0.833
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25,0042300405,1610612738,Boston,Celtics,BOS,celtics,1629004,Svi,Mykhailiuk,S. Mykhailiuk,...,0.000,0.000,0.000,0.000,0.0,0.0,0.0,0.0,0.000,0.000
26,0042300405,1610612738,Boston,Celtics,BOS,celtics,1629674,Neemias,Queta,N. Queta,...,0.000,0.000,0.000,0.000,0.0,0.0,0.0,0.0,0.000,0.000
27,0042300405,1610612738,Boston,Celtics,BOS,celtics,1630531,Jaden,Springer,J. Springer,...,0.000,0.000,0.000,0.000,0.0,0.0,0.0,0.0,0.000,0.000
28,0042300405,1610612738,Boston,Celtics,BOS,celtics,1630214,Xavier,Tillman,X. Tillman,...,0.000,0.000,0.000,0.000,0.0,0.0,0.0,0.0,0.000,0.000


In [61]:
team_performance_list2 = list()

for game_id in all_playoffs_games["GAME_ID"]:
    try:
        team_box_score = boxscorescoringv3.BoxScoreScoringV3(game_id = game_id).get_data_frames()[1]
        team_performance_list2.append(team_box_score)
        time.sleep(2)
    except:
        print("you tried but oh well, row lost i guess")
team_performance_df = pd.concat(team_performance_list2)

team_performance_df

you tried but oh well, row lost i guess
you tried but oh well, row lost i guess


,gameId,teamId,teamCity,teamName,teamTricode,teamSlug,minutes,percentageFieldGoalsAttempted2pt,percentageFieldGoalsAttempted3pt,percentagePoints2pt,...,percentagePointsFastBreak,percentagePointsFreeThrow,percentagePointsOffTurnovers,percentagePointsPaint,percentageAssisted2pt,percentageUnassisted2pt,percentageAssisted3pt,percentageUnassisted3pt,percentageAssistedFGM,percentageUnassistedFGM
0,0041900101,1610612749,Milwaukee,Bucks,MIL,bucks,240:00,0.533,0.467,0.455,...,0.091,0.164,0.127,0.382,0.640,0.360,0.857,0.143,0.718,0.282
1,0041900101,1610612753,Orlando,Magic,ORL,magic,240:00,0.539,0.461,0.459,...,0.098,0.148,0.205,0.311,0.536,0.464,0.875,0.125,0.659,0.341
0,0041900101,1610612749,Milwaukee,Bucks,MIL,bucks,240:00,0.533,0.467,0.455,...,0.091,0.164,0.127,0.382,0.640,0.360,0.857,0.143,0.718,0.282
1,0041900101,1610612753,Orlando,Magic,ORL,magic,240:00,0.539,0.461,0.459,...,0.098,0.148,0.205,0.311,0.536,0.464,0.875,0.125,0.659,0.341
0,0041900102,1610612749,Milwaukee,Bucks,MIL,bucks,240:00,0.539,0.461,0.486,...,0.054,0.108,0.099,0.378,0.444,0.556,0.933,0.067,0.619,0.381
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,0042300404,1610612738,Boston,Celtics,BOS,celtics,240:00,0.488,0.513,0.357,...,0.071,0.143,0.107,0.310,0.533,0.467,0.714,0.286,0.621,0.379
0,0042300405,1610612738,Boston,Celtics,BOS,celtics,240:00,0.562,0.438,0.472,...,0.151,0.160,0.160,0.396,0.560,0.440,0.846,0.154,0.658,0.342
1,0042300405,1610612742,Dallas,Mavericks,DAL,mavericks,240:00,0.526,0.474,0.545,...,0.102,0.080,0.068,0.477,0.500,0.500,0.545,0.455,0.514,0.486
0,0042300405,1610612738,Boston,Celtics,BOS,celtics,240:00,0.562,0.438,0.472,...,0.151,0.160,0.160,0.396,0.560,0.440,0.846,0.154,0.658,0.342


In [63]:
player_performace_df.to_csv("player_performance.csv", index = False)
all_playoffs_games.to_csv('playoffs_games.csv', index=False)
team_performance_df.to_csv('team_performance.csv', index = False)